In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType


@tool
def hash_string(word:str):
    """
    This function to get the hash value of a word.

    :param input_string: The string to be hashed
    :return: The hash value of the input string
    """
    import hashlib
    hash_object = hashlib.sha256()
    hash_object.update(word.encode())
    hash_value = hash_object.hexdigest()
    return hash_value


llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

tools = [hash_string]

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent


AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nhash_string(word: str) - This function to get the hash value of a word.\n\n:param input_string: The string to be hashed\n:return: The hash value of the input string\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [hash_string]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'), llm=Cha

In [6]:
agent.agent.llm_chain.llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11ba8d460>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11ba12b20>, root_client=<openai.OpenAI object at 0x11ba59dc0>, root_async_client=<openai.AsyncOpenAI object at 0x11ba8d490>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [7]:
msg_iterator=agent.stream('Translate the follow word to English, and then write a sentence starting with the word, and then get the hash value of the result, "你好"')

from langchain_core.agents import AgentAction, AgentStep, AgentFinish
from langchain_core.messages import  BaseMessageChunk

first = True
gathered = None
for chunk in msg_iterator:
    print(chunk)
    print(type(chunk))
    print('---', isinstance(chunk, BaseMessageChunk))
    print('---', isinstance(chunk, AgentAction))
    print('---', isinstance(chunk, AgentStep))
    print('---', isinstance(chunk, AgentFinish))
    
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk
gathered




> Entering new None chain...
{'actions': [AgentAction(tool='hash_string', tool_input='Hello, how are you today?', log='The word "你好" is Chinese and translates to "Hello" in English. I will now create a sentence starting with "Hello" and then get the hash value of the result.\nAction: hash_string\nAction Input: "Hello, how are you today?"')], 'messages': [AIMessage(content='The word "你好" is Chinese and translates to "Hello" in English. I will now create a sentence starting with "Hello" and then get the hash value of the result.\nAction: hash_string\nAction Input: "Hello, how are you today?"', additional_kwargs={}, response_metadata={})]}
<class 'langchain_core.runnables.utils.AddableDict'>
--- False
--- False
--- False
--- False
The word "你好" is Chinese and translates to "Hello" in English. I will now create a sentence starting with "Hello" and then get the hash value of the result.
Action: hash_string
Action Input: "Hello, how are you today?"
Observation: 6e766a49e512e0ba0bc935e2aacd

{'actions': [AgentAction(tool='hash_string', tool_input='Hello, how are you today?', log='The word "你好" is Chinese and translates to "Hello" in English. I will now create a sentence starting with "Hello" and then get the hash value of the result.\nAction: hash_string\nAction Input: "Hello, how are you today?"')],
 'messages': [AIMessage(content='The word "你好" is Chinese and translates to "Hello" in English. I will now create a sentence starting with "Hello" and then get the hash value of the result.\nAction: hash_string\nAction Input: "Hello, how are you today?"', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='6e766a49e512e0ba0bc935e2aacd3e5a4a34add17f83afc4c9e669c70241cd48', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I now know the final answer.\nFinal Answer: The hash value of the sentence "Hello, how are you today?" is 6e766a49e512e0ba0bc935e2aacd3e5a4a34add17f83afc4c9e669c70241cd48.', additional_kwargs={}, response_metadata={})],
 'steps

In [22]:
gathered.get('steps')[0].observation

'6e766a49e512e0ba0bc935e2aacd3e5a4a34add17f83afc4c9e669c70241cd48'